<a href="https://colab.research.google.com/github/hewenpang/finetune_path/blob/main/casual_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers==4.45.2
from datasets import Dataset
from transformers import  AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer,DataCollatorForLanguageModeling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 103.7 MB/s eta 0:00:00
  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
ds = Dataset.load_from_disk('/content/gdrive/MyDrive/alpaca_data_zh')
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[0]

{'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
 'input': '',
 'instruction': '保持健康的三个提示。'}

# 数据集预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained('Langboat/bloom-389m-zh')
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

BloomTokenizerFast(name_or_path='Langboat/bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
def process_fn(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [] , [] , []
    instruction = tokenizer('/n'.join(['Human:'+example['instruction'],example['input']]).strip()+'/n/nAssistant:')
    response = tokenizer(example['output']+tokenizer.eos_token)
    input_ids = instruction['input_ids'] + response['input_ids']
    attention_mask = instruction['attention_mask'] + response['attention_mask']
    labels = [-100]* len(instruction['input_ids']) + response['input_ids'] #instruction部分的labels 都是看不到的 仅仅计算output部分的loss
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        'input_ids':input_ids,
        'attention_mask':attention_mask,
        'labels':labels
    }

In [7]:
tokenizered_ds = ds.map(process_fn,remove_columns=ds.column_names)
tokenizered_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [8]:
print(tokenizered_ds[0])

{'input_ids': [23069, 29, 6583, 24772, 8995, 13533, 420, 10839, 10839, 10839, 4122, 15263, 29, 4744, 583, 6583, 24772, 8995, 13533, 1022, 189, 189, 20, 17, 210, 6583, 8416, 3228, 420, 8634, 1900, 13648, 8416, 5625, 355, 1202, 29011, 553, 30355, 1298, 15599, 355, 961, 4872, 34650, 5980, 355, 10915, 15342, 7761, 355, 1403, 11472, 6189, 20465, 671, 189, 21, 17, 210, 20122, 13660, 420, 8634, 13869, 20189, 373, 17070, 553, 16382, 553, 1204, 6165, 1430, 641, 14562, 16130, 24251, 15502, 7984, 355, 7981, 1220, 6538, 553, 1220, 14562, 641, 13545, 10249, 355, 714, 6583, 24772, 13660, 11297, 671, 189, 22, 17, 210, 17672, 16272, 420, 17672, 1063, 13966, 5980, 18688, 355, 30645, 8634, 1638, 7900, 954, 3779, 210, 38858, 17672, 420, 14054, 17672, 11472, 15375, 10891, 355, 4872, 8416, 7442, 355, 1403, 5323, 4001, 16885, 14721, 1249, 420, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
tokenizer.decode(tokenizered_ds['input_ids'][0])

'Human:保持健康的三个提示。/n/n/nAssistant:以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

# 创建模型

In [10]:
model = AutoModelForCausalLM.from_pretrained('Langboat/bloom-389m-zh')

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

In [11]:
args = TrainingArguments(
    output_dir='./chatbot',
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    gradient_accumulation_steps=8,
)

In [12]:
trainer = Trainer(
    model = model,
    args = args,
    tokenizer = tokenizer,
    train_dataset=tokenizered_ds,
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True)
)

# 模型训练

In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,3.177300
20,3.213700
30,3.100600
40,3.138800
50,2.993500
60,2.844900
70,2.874800
80,2.858800
90,2.808400
100,2.917500


TrainOutput(global_step=2517, training_loss=1.7302351757605776, metrics={'train_runtime': 3815.2217, 'train_samples_per_second': 21.119, 'train_steps_per_second': 0.66, 'total_flos': 1.7466950487588864e+16, 'train_loss': 1.7302351757605776, 'epoch': 2.9986597170513773})

In [19]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True)

[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试有很多技巧。以下是一些常见的技巧：\n\n1. 复习计划：考前应有详细的学习计划。复习内容须与应试科目相吻合。\n\n2. 时间管理：考前应制定一个明确的学习时间，避免拖延或分心。要制定复习计划，记录每门功课的时间，以便于自律。\n\n3. 避免压力过大：考前应制定一个合理的作息时间，保证身心都能得到充足的休息。\n\n4. 客观性分析：在备考过程中要注意对所给材料的客观性分析。对于容易出错或难理解的内容，应客观对待，重点关注其中的逻辑推导。\n\n5. 认真准备：准备考前应有充分的心得和知识储备，以便于有效地应对考试。可以查阅相关资料，以便在备考中提供依据。\n\n总之，在备考过程中要保持认真、细致、仔细的态度，努力做到考前充分准备，考后对所学内容有充分的理解和运用。'}]